In [1]:
# Cell 1: (only run if google_play_scraper is not installed)
import sys
!{sys.executable} -m pip install google-play-scraper --quiet
print("google-play-scraper install step completed (or already installed).")


google-play-scraper install step completed (or already installed).



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


1️⃣ 02_scrape_app_reviews.ipynb

Purpose: Scaffold for scraping app reviews from Google Play.

Output: Raw CSV samples for each app (*_reviews_sample.csv).

Status: Done, small sample fetched (20 reviews per app).

In [5]:
# Cell 2: Imports and helper function
from google_play_scraper import reviews, Sort
import pandas as pd
import time
import os

DATA_DIR = "../data"
os.makedirs(DATA_DIR, exist_ok=True)

def fetch_reviews_to_df(app_package: str, n: int = 5, lang: str = "en"):
    """
    Fetch n reviews for app_package and return a DataFrame.
    Note: app_package must be the Android package name (e.g. 'com.example.app').
    """
    all_reviews = []
    count = 0
    # google_play_scraper reviews returns tuples via pagination; use cursor-based fetch
    result, _ = reviews(
        app_package,
        lang=lang,
        country='us',
        sort=Sort.NEWEST,
        count=n
    )
    for r in result:
        all_reviews.append({
            "reviewId": r.get("reviewId"),
            "userName": r.get("userName"),
            "score": r.get("score"),
            "content": r.get("content"),
            "thumbsUpCount": r.get("thumbsUpCount"),
            "at": r.get("at").isoformat() if r.get("at") else None,
            "replyContent": r.get("replyContent"),
            "repliedAt": r.get("repliedAt").isoformat() if r.get("repliedAt") else None
        })
        count += 1
    df = pd.DataFrame(all_reviews)
    return df


In [3]:
app_packages = {
    "Notion": "notion.id",
    "Trello": "com.trello",
    "Slack": "com.Slack"
}



app_packages


{'Notion': 'notion.id', 'Trello': 'com.trello', 'Slack': 'com.Slack'}

In [6]:
# Cell 4: Run a small fetch for each app and save CSVs
n_per_app = 20
for app_name, pkg in app_packages.items():
    print(f"Fetching {n_per_app} reviews for {app_name} ({pkg})...")
    try:
        df = fetch_reviews_to_df(pkg, n=n_per_app)
        out_path = f"{DATA_DIR}/{app_name.lower().replace(' ','_')}_reviews_sample.csv"
        df.to_csv(out_path, index=False)
        print(f"Saved {len(df)} reviews to {out_path}")
    except Exception as e:
        print(f"Error fetching {app_name} ({pkg}):", e)
    # polite delay
    time.sleep(1)
    
print("Done test fetch.")


Fetching 20 reviews for Notion (notion.id)...
Saved 20 reviews to ../data/notion_reviews_sample.csv
Fetching 20 reviews for Trello (com.trello)...
Saved 20 reviews to ../data/trello_reviews_sample.csv
Fetching 20 reviews for Slack (com.Slack)...
Saved 20 reviews to ../data/slack_reviews_sample.csv
Done test fetch.


In [1]:
import os
os.getcwd()


'd:\\PM\\productivity-friction-insight-report\\notebooks'

In [2]:
import os
os.listdir("../data")


['.gitkeep',
 'notion_reviews_sample.csv',
 'slack_reviews_sample.csv',
 'todoist_reviews_sample.csv']

In [8]:
# -------------------------------
# Imports
# -------------------------------
import pandas as pd
import os

# -------------------------------
# File paths
# -------------------------------
# Use absolute path to avoid issues
DATA_DIR = r"D:\PM\productivity-friction-insight-report\data"

# List sample CSV files
csv_files = [f for f in os.listdir(DATA_DIR) if f.endswith("_reviews_sample.csv")]
print("Found CSV files:", csv_files)

# -------------------------------
# Load data
# -------------------------------
# Dictionary to hold dataframes
app_reviews = {}

for f in csv_files:
    app_name = f.replace("_reviews_sample.csv", "")
    df = pd.read_csv(os.path.join(DATA_DIR, f))
    app_reviews[app_name] = df
    print(f"Loaded {len(df)} rows for {app_name}")
    
# Quick peek at one of them
app_reviews[list(app_reviews.keys())[0]].head()

# -------------------------------
# Cleaning steps (example)
# -------------------------------
# Normalize columns and filter relevant fields
for app_name, df in app_reviews.items():
    df.columns = [c.strip().lower() for c in df.columns]
    # Keep only relevant columns if they exist
    relevant_cols = [c for c in ["reviewId", "userName", "score", "content", "at"] if c in df.columns]
    app_reviews[app_name] = df[relevant_cols]
    print(f"{app_name}: cleaned columns -> {app_reviews[app_name].columns.tolist()}")

# -------------------------------
# Save cleaned data back (optional)
# -------------------------------
CLEAN_DIR = os.path.join(DATA_DIR, "cleaned")
os.makedirs(CLEAN_DIR, exist_ok=True)

for app_name, df in app_reviews.items():
    file_path = os.path.join(CLEAN_DIR, f"{app_name}_reviews_cleaned.csv")
    df.to_csv(file_path, index=False)
    print(f"Saved cleaned data for {app_name} to {file_path}")


Found CSV files: ['notion_reviews_sample.csv', 'slack_reviews_sample.csv', 'trello_reviews_sample.csv']
Loaded 20 rows for notion
Loaded 20 rows for slack
Loaded 20 rows for trello
notion: cleaned columns -> ['score', 'content', 'at']
slack: cleaned columns -> ['score', 'content', 'at']
trello: cleaned columns -> ['score', 'content', 'at']
Saved cleaned data for notion to D:\PM\productivity-friction-insight-report\data\cleaned\notion_reviews_cleaned.csv
Saved cleaned data for slack to D:\PM\productivity-friction-insight-report\data\cleaned\slack_reviews_cleaned.csv
Saved cleaned data for trello to D:\PM\productivity-friction-insight-report\data\cleaned\trello_reviews_cleaned.csv
